In [13]:
from src.text_example.load_example_Notre_dame_wikipedia import load_text_a,load_text_b,example_parameters
from src.preprocessing.text_preprocessing import convert_to_list
from src.clustering.cluster_search import find_cluster
from src.web.callbacks.graph_callbacks import create_bubble_plot
import pandas as pd
import plotly.express as px

In [109]:
a = load_text_a()
b = load_text_b()
sep = example_parameters()[0]
#print(sep)
a = convert_to_list(a,sep)
b = convert_to_list(b,sep)
df = find_cluster(a,b,10,'a','b')

2025-03-24 19:50:54,271 - src.clustering.cluster_search - INFO - Length of sequences: 25663 and 29924 items
2025-03-24 19:50:54,272 - src.clustering.cluster_search - INFO - Using minimum cluster length: 10
2025-03-24 19:50:54,453 - src.clustering.cluster_search - INFO - Found 580 clusters


In [110]:
print(df.shape)
create_bubble_plot(df,'a','b')

(580, 6)


In [10]:
df = delete_not_ascending_cluster(df,'a','b')
print(df.shape)
create_bubble_plot(df,'a','b')


NameError: name 'delete_not_ascending_cluster' is not defined

In [10]:
def clean_found_clusters(df):
    return filter_pseudoclusters(df)

In [14]:
df = clean_found_clusters(df)

Initial clusters: 732
Pseudoclusters found (non-ascending start_b): 35
Remaining clusters after filtering: 697


In [7]:
df.to_csv('cluster_df.csv',sep=';',decimal=',')
df.to_parquet('cluster_df.parquet')
df.to_pickle('cluster_df.pickle')

In [ ]:
df

,start_a,end_a,start_b,end_b,length,differenz
0,2,10,28532,28540,8,28530
1,105,124,127,146,19,22
2,225,236,473,484,11,248
3,442,456,585,599,14,143
4,465,475,600,610,10,135
...,...,...,...,...,...,...
727,25197,25220,29361,29384,23,4164
728,25243,25252,28544,28553,9,3301
729,25418,25473,29029,29084,55,3611
730,25505,25518,28619,28632,13,3114


In [1]:
import pandas as pd
import numpy as np
from scipy import stats

def detect_outliers_by_local_consistency(cluster_df, start_a_col='start_a', start_b_col='start_b', 
                                        window_size=5, z_threshold=3.0, verbose=True):
    """
    Detects and filters outliers in start_b values based on local consistency of the relationship
    between start_a and start_b. Uses a sliding window approach to analyze local patterns.
    
    Parameters:
    -----------
    cluster_df : pandas.DataFrame
        DataFrame containing cluster information
    start_a_col : str, default='start_a'
        Column name for the start position in text A
    start_b_col : str, default='start_b'
        Column name for the start position in text B
    window_size : int, default=5
        Size of the sliding window for local consistency analysis
    z_threshold : float, default=3.0
        Z-score threshold for outlier detection
    verbose : bool, default=True
        Whether to print information about filtered clusters
        
    Returns:
    --------
    pandas.DataFrame
        Filtered DataFrame with outliers removed
    """
    # Make a copy of the input DataFrame to avoid modifying the original
    df = cluster_df.copy()
    
    # Initial number of clusters
    initial_count = len(df)
    
    # Make sure the DataFrame is sorted by start_a
    df = df.sort_values(by=start_a_col).reset_index(drop=True)
    
    # Calculate the difference between start_b and start_a
    df['diff'] = df[start_b_col] - df[start_a_col]
    
    # Initialize outlier mask
    is_outlier = np.zeros(len(df), dtype=bool)
    
    # Sliding window approach to detect local outliers
    for i in range(len(df)):
        # Define window bounds (handle edge cases)
        window_start = max(0, i - window_size // 2)
        window_end = min(len(df), i + window_size // 2 + 1)
        
        # Get differences in the current window, excluding the current point
        window_indices = list(range(window_start, window_end))
        if i in window_indices:
            window_indices.remove(i)
        
        if not window_indices:  # Skip if window is empty
            continue
            
        window_diffs = df.loc[window_indices, 'diff'].values
        
        # Skip if window has less than 3 points (not enough for robust statistics)
        if len(window_diffs) < 3:
            continue
        
        # Calculate robust statistics for the window
        median_diff = np.median(window_diffs)
        mad = stats.median_abs_deviation(window_diffs, scale=1.4826)  # Consistent with normal distribution
        
        # Handle case where MAD is 0 (all points in window have same diff)
        if mad == 0:
            # Use a small default value
            mad = np.std(window_diffs) if np.std(window_diffs) > 0 else 1.0
        
        # Calculate z-score for the current point
        current_diff = df.loc[i, 'diff']
        z_score = abs(current_diff - median_diff) / mad
        
        # Mark as outlier if z-score exceeds threshold
        if z_score > z_threshold:
            is_outlier[i] = True
    
    # Count and remove outliers
    outlier_count = np.sum(is_outlier)
    filtered_df = df[~is_outlier].copy()
    
    # Drop the temporary column
    filtered_df = filtered_df.drop('diff', axis=1)
    
    # Print summary if verbose
    if verbose:
        print(f"Initial clusters: {initial_count}")
        print(f"Outliers detected: {outlier_count}")
        print(f"Remaining clusters after filtering: {len(filtered_df)}")
        
        if outlier_count > 0:
            # Show some examples of detected outliers
            outlier_examples = df[is_outlier].head(min(5, outlier_count))
            print("\nExample outliers:")
            for idx, row in outlier_examples.iterrows():
                print(f"Row {idx}: start_a={row[start_a_col]}, start_b={row[start_b_col]}, " 
                      f"diff={row['diff']}")
    
    return filtered_df, outlier_count

In [2]:
def is_column_ascending(df, column_name):
    values = df[column_name]
    return values.equals(values.sort_values(ignore_index=False))

In [ ]:
def delete_not_ascending_cluster(df, text_a_name='a', text_b_name='b'):
    # Errorhandling

    column_name_a= f'start_{text_a_name}'
    column_name_b= f'start_{text_b_name}'

    if df.empty:
        raise ValueError("DataFrame is empty")   
     
    if column_name_a not in df.columns:
        raise KeyError(f"Column '{column_name_a}' not found in DataFrame")
    if column_name_b not in df.columns:
        raise KeyError(f"Column '{column_name_b}' not found in DataFrame")
    
    df = remove_overlapping_clusters(df,column_name_a,f'end_{text_a_name}')

    z_thresh = [4,3.5,3,2.8,2.5,2,2,1.8,1.5,1]
    win_size = [15,15,13,11,9,20,7,5,15,3]
    parameter_set = 0
    while not is_column_ascending(df,column_name_b):
        df, outlier_count = detect_outliers_by_local_consistency(df,column_name_a,column_name_b,win_size[parameter_set],z_thresh[parameter_set])
        if outlier_count == 0:
            parameter_set += 1
            if parameter_set == len(z_thresh):
                print('not all wrong clusters removed')
                return df
    return df

In [57]:
is_column_ascending(df,'start_a')

True

In [73]:
is_column_ascending(df,'start_b')

False

In [76]:
last = 0
for i in df['start_b'].values:
    print(i)
    if i <last:
        print('_________________')
    last = i

1031
1113
1128
1140
1159
1277
1289
1468
1479
1656
1677
1703
1720
1758
1791
2283
2292
2360
2419
2485
2567
2633
2784
2793
2802
2831
2840
2854
2867
3087
3131
3154
3688
3914
3928
3937
3962
3972
3997
4028
4248
4258
4274
4296
4360
4376
4400
4423
4441
4468
4692
4719
4735
5062
5071
5121
5163
5179
5288
5317
5341
5371
5413
5426
5445
5468
5501
5524
5587
5627
5659
6627
6648
6682
6706
6721
6767
6815
6889
7577
7595
7610
7657
7766
7817
7890
8010
8368
8523
8548
8573
8584
8593
8631
8663
8731
8768
8791
8814
8846
8870
10072
10081
10092
10101
10125
10214
10224
10305
10331
10341
10357
10395
10427
10436
10485
10504
10694
10729
10747
10757
11825
11825
11847
11847
11902
11937
11953
12000
12020
12041
12089
12115
12132
12198
12230
12247
12272
12282
12272
_________________
12282
12354
12544
12558
12580
12599
12608
12638
12667
12687
12696
12732
12792
12809
12825
12853
12875
13046
13059
13071
13080
13156
13351
13368
13402
13426
13469
13809
13835
13913
13965
13978
14187
14201
14220
14269
14297
14347
14366
14393
144

In [83]:
df[df['start_b'].isin([12247,12272,12282,12354])]

,start_a,end_a,start_b,end_b,length,differenz
261,7751,7776,12247,12272,25,4496
264,7812,7880,12272,12340,68,4460
269,7938,7960,12354,12376,22,4416


In [3]:
def remove_overlapping_clusters(cluster_df, start_a_col='start_a', end_a_col='end_a', verbose=True):
    """
    Removes rows where start_a is smaller than the end_a of the previous row.
    This ensures that clusters do not overlap in text A.
    
    Parameters:
    -----------
    cluster_df : pandas.DataFrame
        DataFrame containing cluster information
    start_a_col : str, default='start_a'
        Column name for the start position in text A
    end_a_col : str, default='end_a'
        Column name for the end position in text A
    verbose : bool, default=True
        Whether to print information about removed rows
        
    Returns:
    --------
    pandas.DataFrame
        Filtered DataFrame with overlapping clusters removed
    """
    # Make a copy of the input DataFrame to avoid modifying the original
    df = cluster_df.copy()
    
    # Initial number of clusters
    initial_count = len(df)
    
    # Make sure the DataFrame is sorted by start_a
    df = df.sort_values(by=start_a_col).reset_index(drop=True)
    
    # Initialize mask for valid rows (non-overlapping)
    valid_rows = pd.Series(True, index=df.index)
    
    # Previous end_a value
    prev_end_a = None
    
    # Check each row for overlap with previous
    for idx in df.index:
        current_start_a = df.loc[idx, start_a_col]
        
        # Skip first row
        if prev_end_a is not None:
            # If current start_a is smaller than previous end_a, mark as invalid
            if current_start_a < prev_end_a:
                valid_rows[idx] = False
        
        # Update previous end_a
        prev_end_a = df.loc[idx, end_a_col]
    
    # Apply the mask to filter out overlapping rows
    filtered_df = df[valid_rows].copy()
    
    # Count removed rows
    removed_count = initial_count - len(filtered_df)
    
    # Print summary if verbose
    if verbose:
        print(f"Initial clusters: {initial_count}")
        print(f"Overlapping clusters removed: {removed_count}")
        print(f"Remaining clusters: {len(filtered_df)}")
        
    return filtered_df

# Example usage:
# filtered_df = remove_overlapping_clusters(cluster_df)

In [82]:
df=remove_overlapping_clusters(df)

Initial clusters: 732
Overlapping clusters removed: 5
Remaining clusters: 727


In [16]:
def detect_outliers(df,col,window_size):
    df[col] = df[col].rolling(window_size).apply(lambda x: np.mean(x))

,start_a,end_a,start_b,end_b,length,differenz
0,105,124,127,146,19,22
1,225,236,473,484,11,248
2,442,456,585,599,14,143
3,465,475,600,610,10,135
4,507,535,962,990,28,455
...,...,...,...,...,...,...
575,25145,25159,29296,29310,14,4151
576,25168,25178,29319,29329,10,4151
577,25197,25220,29361,29384,23,4164
578,25418,25473,29029,29084,55,3611


In [25]:
import numpy as np
df['roll'] = df['start_b'].rolling(2).apply(lambda x: 0 if (x[0]< x[1]) else 1)

KeyError: 0

In [505]:
df['diff'] = df['start_b'].diff()
#df['flag'] = df['diff'].apply(lambda x: 1 if x<1 else 0)
df['flag'] = 0
neg_diffs = df['diff'] < 1
df.loc[neg_diffs, 'flag'] = 1
shifted_mask = pd.Series(neg_diffs).shift(-1).fillna(False)
df.loc[shifted_mask, 'flag'] = 1

C:\Users\rdeic\AppData\Local\Temp\ipykernel_18116\3377497721.py:6: FutureWarning:

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`



In [508]:
df.head(50)

,start_a,end_a,start_b,end_b,length,differenz,diff,flag
0,105,124,127,146,19,22,NaN,0
1,225,236,473,484,11,248,346.0,0
2,442,456,585,599,14,143,112.0,0
3,465,475,600,610,10,135,15.0,0
4,507,535,962,990,28,455,362.0,0
5,543,572,998,1027,29,455,36.0,0
6,577,598,1031,1052,21,454,33.0,0
7,601,627,1052,1078,26,451,21.0,0
8,659,673,1113,1127,14,454,61.0,0
9,776,787,1277,1288,11,501,164.0,0


In [503]:
a = load_text_a()
b = load_text_b()
sep = example_parameters()[0]
#print(sep)
a = convert_to_list(a,sep)
b = convert_to_list(b,sep)
df = find_cluster(a,b,10,'a','b')

2025-03-24 21:49:57,984 - src.clustering.cluster_search - INFO - Length of sequences: 25663 and 29924 items
2025-03-24 21:49:57,985 - src.clustering.cluster_search - INFO - Using minimum cluster length: 10
2025-03-24 21:49:58,199 - src.clustering.cluster_search - INFO - Found 580 clusters


In [529]:
df['diff'] = df['start_b'].diff()
#df['flag'] = df['diff'].apply(lambda x: 1 if x<1 else 0)
df['flag'] = 0
neg_diffs = df['diff'] < 0
df.loc[neg_diffs, 'flag'] = 1
shifted_mask = pd.Series(neg_diffs).shift(-1).fillna(False)
df.loc[shifted_mask, 'flag'] = 1

remove_indexes = np.zeros(df.shape[0])
last_row_2 = ''
last_row = ''
for i in df.iterrows():
    #print(f'{i[1]}\t')
    if last_row == '' and i[1]['flag']==1:
        remove_indexes[i[0]] = True
    
    elif last_row == '' and i[1]['flag']==0:
        remove_indexes[i[0]] = False

    
    elif last_row[1]['flag'] == 1 and i[1]['flag'] == 1:

        if last_row_2 == '':
            remove_indexes[i[0]] = True

        elif i[1]['start_b']>last_row_2[1]['start_b']:

            print(f'first {i[0]}')
            remove_indexes[i[0]] = False
            remove_indexes[last_row[0]] = True
        else:
            print(f'second {i[0]}')
            remove_indexes[i[0]] = True       

        
#    elif i[1]['diff']>0 and last_row[1]['flag'] == 1 and i[1]['flag'] == 1:
#        print(f'second {i[0]}')
#        remove_indexes[i[0]-1] = True
#        remove_indexes[i[0]] = False

 #   elif i[1]['diff']<=0 and last_row[1]['flag'] == 1:
 #       remove_indexes[i[0]] = True

    elif i[1]['flag']==0:
        remove_indexes[i[0]] = False

#    elif last_row[1]['flag'] == 0 and i[1]['flag'] == 1:
#        if i[1]['diff']*-0.2 > last_row[1]['diff']:
#            remove_indexes[i[0]] = True
#        remove_indexes[i[0]] = False

    else:
        remove_indexes[i[0]] = False

    last_row_2 = last_row
    last_row = i


drop_indices = np.where(remove_indexes == 1)[0]

# Methode 1: Mit drop und Index-Labels
df = df.drop(index=drop_indices)
df.reset_index(inplace=True,drop=True)
df['diff'] = df['start_b'].diff()
#df['flag'] = df['diff'].apply(lambda x: 1 if x<1 else 0)
df['flag'] = 0
neg_diffs = df['diff'] < 0
df.loc[neg_diffs, 'flag'] = 1
shifted_mask = pd.Series(neg_diffs).shift(-1).fillna(False)
df.loc[shifted_mask, 'flag'] = 1
#print(remove_indexes)

C:\Users\rdeic\AppData\Local\Temp\ipykernel_18116\528510280.py:6: FutureWarning:

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`

C:\Users\rdeic\AppData\Local\Temp\ipykernel_18116\528510280.py:69: FutureWarning:

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`



In [293]:
df.reset_index(drop=True)

,start_a,end_a,start_b,end_b,length,differenz,diff,flag
0,105,124,127,146,19,22,NaN,0
1,225,236,473,484,11,248,346.0,0
2,442,456,585,599,14,143,112.0,0
3,465,475,600,610,10,135,15.0,0
4,507,535,962,990,28,455,362.0,0
...,...,...,...,...,...,...,...,...
533,25030,25118,28299,28387,88,3269,40.0,0
534,25123,25144,28392,28413,21,3269,93.0,0
535,25145,25159,29296,29310,14,4151,904.0,0
536,25168,25178,29319,29329,10,4151,23.0,0


In [191]:
drop_indices = np.where(remove_indexes == 1)[0]

# Methode 1: Mit drop und Index-Labels
df = df.drop(index=drop_indices)

df['diff'] = df['start_b'].diff()
#df['flag'] = df['diff'].apply(lambda x: 1 if x<1 else 0)
df['flag'] = 0
neg_diffs = df['diff'] < 1
df.loc[neg_diffs, 'flag'] = 1
shifted_mask = pd.Series(neg_diffs).shift(-1).fillna(False)
df.loc[shifted_mask, 'flag'] = 1


df.shape

C:\Users\rdeic\AppData\Local\Temp\ipykernel_18116\2959704183.py:11: FutureWarning:

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`



(516, 8)

In [167]:
df.reset_index(inplace=True)

ValueError: cannot insert level_0, already exists

In [157]:
remove_indexes

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0.,
       1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0.,
       0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1.,
       0., 1., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0.,
       0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 1., 1., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

In [465]:
fig = px.line(df, y='start_b')
fig

In [426]:
fig = px.line(df, y='diff')
fig

In [530]:
fig = px.scatter(df,x='start_a', y='start_b',color='flag')
fig

In [242]:
df[['diff','flag']].iloc[290:330]

,diff,flag
302,38.0,0
303,20.0,0
304,103.0,0
305,38.0,0
306,18.0,0
307,20.0,0
308,11.0,0
309,17.0,0
310,277.0,1
312,-17441.0,1


In [108]:
df.shape

(551, 8)

In [107]:
df.head(50)

,start_a,end_a,start_b,end_b,length,differenz,diff,flag
0,105,124,127,146,19,22,NaN,0
1,225,236,473,484,11,248,346.0,0
2,442,456,585,599,14,143,112.0,0
3,465,475,600,610,10,135,15.0,0
4,507,535,962,990,28,455,362.0,0
5,543,572,998,1027,29,455,36.0,0
6,577,598,1031,1052,21,454,33.0,0
7,601,627,1052,1078,26,451,21.0,0
8,659,673,1113,1127,14,454,61.0,0
9,759,776,14820,14837,17,14061,13707.0,1
